In [2]:
import requests
from bs4 import BeautifulSoup
import pymysql
from selenium import webdriver
import time


headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.67"}

# res = requests.get(url, headers = headers)
# db 연결
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='goodo011@', db='naver_news', charset='utf8')
cursor = db.cursor()

cursor.execute('drop table if exists headlines');
cursor.execute('drop table if exists detail_news');

sql = """
create table headlines
    (
    ord_id varchar(20) not null primary key,
    title varchar(300) not null,
    press varchar(100) not null,  
    categori varchar(20) not null,
    url varchar(300) not null
    );
"""
cursor.execute(sql)

sql = """    
create table detail_news
	(
	ord_id varchar(20) not null primary key,
    reporter varchar(20) not null,
    date varchar(30),
    comment int unsigned,
    text longtext
    );
"""
cursor.execute(sql)
    
# db.commit()
# db.close()


0

In [3]:
what_date = input('어느 일자의 기사들을 보시겠습니까 ("0720" 과 같이 입력하여 주십시오.)').strip()
url = "https://news.naver.com/main/ranking/popularDay.naver?date=2021"+str(what_date)

print(url)

# selenium
browser = webdriver.Chrome()
browser.get(url)


어느 일자의 기사들을 보시겠습니까 ("0720" 과 같이 입력하여 주십시오.)0720
https://news.naver.com/main/ranking/popularDay.naver?date=20210720


In [4]:
soup = BeautifulSoup(browser.page_source, 'lxml')
rankingnews = soup.find_all('div', {'class' : 'rankingnews_box'})

try:
    want_thing = input('정치, 경제, 사회, 생활, 세계, IT 중에 보고싶은 카테고리를 입력하세요.(모든 카테고리를 희망한다면 "모두"를 입력하세요) : ')
    what_press = input('어느 언론사의 기사들을 보시겠습니까. (모든 언론사를 원하신다면 "모두" 라고 입력하십시오)')
    cnt = int(input('각 언론사에서 몇개의 기사들을 보시겠습니까. (1~5의 값을 입력해주세요)'))
    
    for ranking in rankingnews:
        # 언론사
        press = ranking.find('strong', {'class' : 'rankingnews_name'}).text
        # 언론사 안에 뉴스 
        ranks = ranking.find_all('li')
        count = 0
        
        for rank in ranks:
            
            #  언론사가 입력된 input 안에 없거나 '모두' 가 아니라면 
            if what_press not in press and what_press != '모두':
                continue
            count+=1
            
            #처음에 모두를 입력하면 
            if count > cnt:
                break;
            link = rank.find('a' , {'class' : "list_title nclicks('RBP.rnknws')"})
             
            if link:
                title = link.get_text()
                link = 'https://news.naver.com'+link['href']            
                ord_id = link.split('&')[3] + link.split('&')[4]
            
                print(f'link >>>> {link} : title >>>{title} :  id >>>> {id}')
#         text_title = i.get_text()
#         oid = i['href'].split('&')[-3].split('=')[1]
#         aid = i['href'].split('&')[-2].split('=')[1]
#         # id값
#         id = oid+ aid

                # 상세페이지로
                url = link
                browser.get(url)
                # 상세 페이지 soup
                soup2 = BeautifulSoup(browser.page_source, 'lxml')
                
                # 카테고리
                categori = soup2.find('em' , {'class' : 'guide_categorization_item'})
                
                print(categori, want_thing)
                
                if categori and want_thing == categori.text:
                    categori = categori.text
                
                elif want_thing == ('모두'):
                    catrgori = categori.text
                
                else:
                    continue
                
                title = title.replace('"', "'")

#                 sql = """insert into headlines (ord_id, title, press, categori, link)
#                 values( %d, %s, %s, %s, %s);
#                 """
#                 time.sleep(2)
#                 cursor.execute(sql)

                # 기자
                reporter = soup2.find('div', {'class' : 'journalistcard_summary_name'})
                if reporter:
                    reporter = reporter.text
                    
                # 게시 날짜
                date = soup2.find('span', {'class' : 't11'}).text
                
                #comment_cnt = browser.find_element_by_css_selector('span.u_cbox_info_txt')
                #print(comment_cnt.get_attribute(''))
                
                # 기사본문
                texts = soup2.find('div', {'id' : 'articleBodyContents'})

                soup2.select('li > strong')
                texts = texts.text # 기사 본문
                keyword = keyword_search(texts)
                print(ord_id, reporter, date, keyword)
                
                sql = f"""insert into detail_news values("{ord_id}", "{reporter}", "{date}", '{keyword}');"""
                cursor.execute(sql)
                time.sleep(2)           
            else:
                continue;
finally:
    db.commit()
    db.close()

정치, 경제, 사회, 생활, 세계, IT 중에 보고싶은 카테고리를 입력하세요.(모든 카테고리를 희망한다면 "모두"를 입력하세요) : 경제
어느 언론사의 기사들을 보시겠습니까. (모든 언론사를 원하신다면 "모두" 라고 입력하십시오)모두
각 언론사에서 몇개의 기사들을 보시겠습니까. (1~5의 값을 입력해주세요)4
link >>>> https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=001&aid=0012535673&rankingType=RANKING : title >>>[르포] 지하철서 '노 마스크'…일 20만명 확진 전망에도 규제 푼 영국 :  id >>>> <built-in function id>
<em class="guide_categorization_item">세계</em> 경제
link >>>> https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=001&aid=0012537866&rankingType=RANKING : title >>>11시간만에 구조된 김홍빈, 의식 있었다…구조중 줄 끊기며 추락 :  id >>>> <built-in function id>
<em class="guide_categorization_item">사회</em> 경제
link >>>> https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=001&aid=0012536588&rankingType=RANKING : title >>>트로트 가수 김호중, 폭행 혐의로 경찰 입건 :  id >>>> <built-in function id>
<em class="guide_categorization_item">사회</em> 경제
link >>>> https://news.naver.com/main/ranking/read.naver?mode=LS

NameError: name 'keyword' is not defined

In [ ]:
def keyword(arr)

In [1]:
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='naver_news', charset='utf8')
cursor = db.cursor()

soup = BeautifulSoup(browser.page_source, 'lxml')
rankingnews = soup.find_all('div', {'class' : 'rankingnews_box'})
# print(browser.page_source)

for ranking in rankingnews:
        # 언론사
        press = ranking.find('strong', {'class' : 'rankingnews_name'}).text
        # 언론사 안에 뉴스 
        ranks = ranking.find_all('li')
        count = 0
        
        for rank in ranks:
            link = rank.find('a' , {'class' : "list_title nclicks('RBP.rnknws')"})
          
            title = link.get_text()
            link2 = 'https://news.naver.com'+link['href']            
            
            oid = link['href'].split('&')[-3].split('=')[1]
            aid = link['href'].split('&')[-2].split('=')[1]
            ord_id = oid + aid 
            
            print(ord_id)
            title = title.replace('"', "'")
            print(f'link >>>> {link} : title >>>{title} :  id >>>> {id}')
            
            sql = f"""insert into headlines (ord_id, title, press, url)
                values({ord_id}, {title}, {press}, {link2});"""
            time.sleep(1)
            cursor.execute(sql)
            
            db.commit()
              
   
 

SyntaxError: f-string: empty expression not allowed (<ipython-input-1-37b532de2cdc>, line 29)

In [ ]:
# soup = BeautifulSoup(res.text, "lxml")

# news_items = soup.find_all("div", {"class" : "rankingnews_box"})

# # len(news_items)

# for items in news_items:

    
#     # 신문사 이름
#     name = items.find("strong", {"class" : "rankingnews_name"})
#     # 신문사 해당 뉴스 list
# #     news_list = items.find("ul", {"class" : "rankingnews_list"})
# #     news_list = items.ul.children
#     news_list = items.select('.rankingnews_list')
#     for i in news_list:
#         print(i)
    
#     # 신문사 해당 list 뉴스 정보
#     news_test = news_list.find_all("a",{"class":"list_title nclicks('RBP.rnknws')"})
    
#     for i in news_test:
        
#         print(i)
#         # 링크
#         link_text = 'https://news.naver.com'+i['href']
#         # title   
#         text_title = i.get_text()
#         oid = i['href'].split('&')[-3].split('=')[1]
#         aid = i['href'].split('&')[-2].split('=')[1]
#         # id값
#         id = oid+ aid
       
#         res2 = requests.get(link_text, headers = headers)
#         res2.raise_for_status()
#         soup = BeautifulSoup(res2.text, "lxml")
#         # 카테고리
#         categori = soup.find('em',{'class':'guide_categorization_item'})
#         print(categori.get_text())
        
        